In [12]:
import os
import glob
import csv
import pandas as pd
import shutil
import snowflake.connector
import configparser
import fnmatch
from configparser import ConfigParser

In [13]:
# Read configuration from properties file
# config = configparser.ConfigParser()
# config.read('config.properties')

# from configparser import ConfigParser

# parser = ConfigParser()
# parser.read('SNOWFLAKE_CREDS.cfg')
# account = parser.get('my_api','account')
# user = parser.get('my_api','user')
# password = parser.get('my_api','password')
# database = parser.get('my_api','database')
# schema = parser.get('my_api','schema')

# connection_parameters = {
#             'account':account,
#             'user':user,
#             'password':password,
#             'database':database,
#             'SCHEMA':schema
#         }
# from snowflake.snowpark import Session

# snow_session = Session.builder.configs(connection_parameters).create()

[]

In [14]:
import time

def put_file(stage,path):
    
    
    path_to_stage = "".join(path.split('/')[:-1])
    print(f"snow_session.file.put(r'{path}', '@{stage}/{path_to_stage}')")
#     snow_session.file.put(rf'{path}', f'@{stage}/{path_to_stage}',auto_compress=False)
#     time.sleep(5)
    curr.execute(f"PUT 'file://{path}' '@{stage}/{path_to_stage}'")
    stage_path = stage+'/'+path
    return stage_path

In [15]:
def create_table(curs,target_table,file_path,delimiter=',',replace=False):
    file_format = f"CREATE OR REPLACE FILE FORMAT my_csv_format TYPE = 'CSV' FIELD_DELIMITER = '{delimiter}'  PARSE_HEADER = TRUE;"
    print(file_format)
    curs.execute(file_format)
    statement = ' OR REPLACE TABLE' if replace else "TABLE IF NOT EXISTS "
    query = f"""
                CREATE {statement} {target_table}
                USING TEMPLATE (
                       SELECT ARRAY_AGG(OBJECT_CONSTRUCT(*))
                       WITHIN GROUP (ORDER BY ORDER_ID)
                       FROM TABLE(
                           INFER_SCHEMA(
                                 LOCATION=>'{file_path}',
                                 FILE_FORMAT=>'my_csv_format'
                                       )
                                )
                            );"""
    print(query)
    curs.execute(query)
    print(f'\n{target_table.upper()} has been created\n')
    

In [5]:
def ingest_csv_file(stage_name,csv_file_path, table_name,skip_header = 1,field_delimiter = ',', target_operation = 'append', *args):
   
    # Connect to Snowflake using the snowflake_config dictionary
    parser = ConfigParser()
    parser.read('SNOWFLAKE_CREDS.cfg')
    account = parser.get('my_api','account')
    user = parser.get('my_api','user')
    password = parser.get('my_api','password')
    database = parser.get('my_api','database')
    schema = parser.get('my_api','schema')
    role = parser.get('my_api','role')
    warehouse = parser.get('my_api','warehouse')
    
    conn = snowflake.connector.connect( user = user ,password = password,account = account ,warehouse = warehouse ,
                                       database = database,schema = schema,role = role)
      
    curs = conn.cursor()
    
    stage_path = put_file(stage = stage_name,path = csv_file_path)
    
    # append or overwrite
    if target_operation.casefold() == 'overwrite':
        print(f"{table_name.upper()} is truncated")
        truncate_query = f"TRUNCATE TABLE {table_name.upper()}"
        curs.execute(truncate_query)
    
    create_table(curs,table_name,stage_path,delimiter=field_delimiter,replace=False)
    # Copy CSV file data into Snowflake table
    copy_query = f"""
                        COPY INTO {table_name.upper()} 
                        FROM '@{database}.{schema}.{stage_path}'
                        FILE_FORMAT = (
                                TYPE = CSV   
                                SKIP_HEADER = {skip_header}
                                FIELD_DELIMITER = '{field_delimiter}')
                        on_error = continue;"""
    print(copy_query)
    curs.execute(copy_query)

    curs.close()
    conn.close()


In [16]:
# def main(csv_file_path):

stage_name = 'internal_stage'
csv_file_path = 'inbound/'
archive_folder = 'archive/'
missing_folder = 'missing/'
failed = 'failed/'

df = pd.read_csv('mapping/metadata1.csv',header = 0)

for file in os.listdir(csv_file_path):
    matching = False
    for ind in df.index:
        if fnmatch.fnmatch(file, df['src_file_prefix'][ind]+'*.csv'):
            if df['is_active'][ind] == 'Y':
                delim = ','
                if df['src_file_delim'][ind] != ',':
                    delim = df['src_file_delim'][ind]
                ingest_csv_file(stage_name,csv_file_path+file,df['tgt_table'][ind],field_delimiter = delim)
#                 print(f"ingest_csv_file({stage_name},{csv_file_path+file},{df['tgt_table'][ind]})")
                shutil.move(csv_file_path+file,'archive/')

            else:
                print(f'Skipping CSV file {file} for table {df["tgt_table"][ind]} due to inactive flag.')
                # Move the skipped file to the missing mapping folder
                shutil.move(csv_file_path + file, failed)
                print(f'CSV file "{file}" moved to missing mapping folder "{failed}".')
            matching = True
            break
    if not matching:
            print(f'\n No file name pattern found for CSV file "{file}" . Skipping the file.\n')
            shutil.move(csv_file_path + file, missing_folder)
            print(f'\nCSV file "{file}" moved to missing mapping folder {missing_folder}".\n')

snow_session.file.put(r'inbound/data_85.csv', '@internal_stage/inbound')


NameError: name 'curr' is not defined

In [11]:
main(r'inbound/')

snow_session.file.put(r'inbound/data_85.csv', '@internal_stage/inbound')


NameError: name 'curr' is not defined

In [7]:
class Metaframework:
    
    def __init__(self,path):
        self.path = path
        parser = ConfigParser()
        parser.read(rf'{self.path}\SNOWFLAKE_CREDS.cfg')
        account = parser.get('my_api','account')
        user = parser.get('my_api','user')
        password = parser.get('my_api','password')
        database = parser.get('my_api','database')
        schema = parser.get('my_api','schema')
        role = parser.get('my_api','role')
        warehouse = parser.get('my_api','warehouse')

        conn = snowflake.connector.connect( user = user ,password = password,account = account ,warehouse = warehouse ,
                                           database = database,schema = schema,role = role)
        
        conn.cursor().execute('create table asshole (id int);')
        
    def config(self):
        pass
        
    def create_table(self,curs,target_table,file_path,delimiter=',',replace=False):
        file_format = f"CREATE OR REPLACE FILE FORMAT my_csv_format TYPE = 'CSV' FIELD_DELIMITER = '{delimiter}'  PARSE_HEADER = TRUE;"
        print(file_format)
        curs.execute(file_format)
        statement = ' OR REPLACE TABLE' if replace else "TABLE IF NOT EXISTS "
        query = f"""
                    CREATE {statement} {target_table}
                    USING TEMPLATE (
                           SELECT ARRAY_AGG(OBJECT_CONSTRUCT(*))
                           WITHIN GROUP (ORDER BY ORDER_ID)
                           FROM TABLE(
                               INFER_SCHEMA(
                                     LOCATION=>'{file_path}',
                                     FILE_FORMAT=>'my_csv_format'
                                           )
                                    )
                                );"""
        print(query)
        curs.execute(query)
        print(f'\n{target_table.upper()} has been created\n')
    
    def put_file(self,stage,path):
    
    
        path_to_stage = "".join(path.split('/')[:-1])
        print(f"snow_session.file.put(r'{path}', '@{stage}/{path_to_stage}')")
    #     snow_session.file.put(rf'{path}', f'@{stage}/{path_to_stage}',auto_compress=False)
    #     time.sleep(5)
        curr.execute(f"PUT 'file://{path}' '@{stage}/{path_to_stage}'")
        stage_path = stage+'/'+path
        return stage_path
        
        
    def ingest_csv_file(self,stage_name,csv_file_path, table_name,skip_header = 1,field_delimiter = ',', target_operation = 'append', *args):
   
        # Connect to Snowflake using the snowflake_config dictionary
        parser = ConfigParser()
        parser.read('SNOWFLAKE_CREDS.cfg')
        account = parser.get('my_api','account')
        user = parser.get('my_api','user')
        password = parser.get('my_api','password')
        database = parser.get('my_api','database')
        schema = parser.get('my_api','schema')
        role = parser.get('my_api','role')
        warehouse = parser.get('my_api','warehouse')

        conn = snowflake.connector.connect( user = user ,password = password,account = account ,warehouse = warehouse ,
                                           database = database,schema = schema,role = role)

        curs = conn.cursor()

        stage_path = put_file(stage = stage_name,path = csv_file_path)

        # append or overwrite
        if target_operation.casefold() == 'overwrite':
            print(f"{table_name.upper()} is truncated")
            truncate_query = f"TRUNCATE TABLE {table_name.upper()}"
            curs.execute(truncate_query)

        create_table(curs,table_name,stage_path,delimiter=field_delimiter,replace=False)
        # Copy CSV file data into Snowflake table
        copy_query = f"""
                            COPY INTO {table_name.upper()} 
                            FROM '@{database}.{schema}.{stage_path}'
                            FILE_FORMAT = (
                                    TYPE = CSV   
                                    SKIP_HEADER = {skip_header}
                                    FIELD_DELIMITER = '{field_delimiter}')
                            on_error = continue;"""
        print(copy_query)
        curs.execute(copy_query)

        curs.close()
        conn.close()

        
    def main(self):
        stage_name = 'internal_stage'
        csv_file_path = 'inbound/'
        archive_folder = 'archive/'
        missing_folder = 'missing/'
        failed = 'failed/'

        df = pd.read_csv('mapping/metadata1.csv',header = 0)

        for file in os.listdir(self.csv_file_path):
            matching = False
            for ind in df.index:
                if fnmatch.fnmatch(file, df['src_file_prefix'][ind]+'*.csv'):
                    if df['is_active'][ind] == 'Y':
                        delim = ','
                        if df['src_file_delim'][ind] != ',':
                            delim = df['src_file_delim'][ind]
                        ingest_csv_file(stage_name,self.csv_file_path+file,df['tgt_table'][ind],field_delimiter = delim)
        #                 print(f"ingest_csv_file({stage_name},{csv_file_path+file},{df['tgt_table'][ind]})")
                        shutil.move(csv_file_path+file,'archive/')

                    else:
                        print(f'Skipping CSV file {file} for table {df["tgt_table"][ind]} due to inactive flag.')
                        # Move the skipped file to the missing mapping folder
                        shutil.move(csv_file_path + file, failed)
                        print(f'CSV file "{file}" moved to missing mapping folder "{failed}".')
                    matching = True
                    break
            if not matching:
                    print(f'\n No file name pattern found for CSV file "{file}" . Skipping the file.\n')
                    shutil.move(csv_file_path + file, missing_folder)
                    print(f'\nCSV file "{file}" moved to missing mapping folder {missing_folder}".\n')

In [8]:
Metaframework(r"D:\snowpark")

ProgrammingError: 002002 (42710): SQL compilation error:
Object 'ASSHOLE' already exists.

In [33]:
snow_session.file.put(r'inbound/sales_2023.csv', '@internal_stage/inbound',auto_compress=False,overwrite = True)

[PutResult(source='sales_2023.csv', target='sales_2023.csv', source_size=5003, target_size=5008, source_compression='NONE', target_compression='NONE', status='UPLOADED', message='')]

In [18]:
for i in range(1,32):
    print(f'col_{i}',end = '|')

col_1|col_2|col_3|col_4|col_5|col_6|col_7|col_8|col_9|col_10|col_11|col_12|col_13|col_14|col_15|col_16|col_17|col_18|col_19|col_20|col_21|col_22|col_23|col_24|col_25|col_26|col_27|col_28|col_29|col_30|col_31|

In [41]:
create_table(curs,'target_table4','@internal_stage/inbound/sales_2023.csv',delimiter = '|',replace = True)

CREATE OR REPLACE FILE FORMAT my_csv_format TYPE = 'CSV' FIELD_DELIMITER = '|'  PARSE_HEADER = TRUE;

                CREATE  OR REPLACE TABLE target_table4
                USING TEMPLATE (
                       SELECT ARRAY_AGG(OBJECT_CONSTRUCT(*))
                       WITHIN GROUP (ORDER BY ORDER_ID)
                       FROM TABLE(
                           INFER_SCHEMA(
                                 LOCATION=>'@internal_stage/inbound/sales_2023.csv',
                                 FILE_FORMAT=>'my_csv_format'
                                       )
                                )
                            );
TARGET_TABLE4 has been created


In [7]:
parser = ConfigParser()
parser.read('SNOWFLAKE_CREDS.cfg')
account = parser.get('my_api','account')
user = parser.get('my_api','user')
password = parser.get('my_api','password')
database = parser.get('my_api','database')
schema = parser.get('my_api','schema')
role = parser.get('my_api','role')
warehouse = parser.get('my_api','warehouse')
conn = snowflake.connector.connect( user = user ,password = password,account = account ,warehouse = warehouse ,
                                       database = database,schema = schema,role = role)
      
curs = conn.cursor()

In [40]:
create_table(curs,'target_table5','@internal_stage/inbound/data_85.csv')

CREATE OR REPLACE FILE FORMAT my_csv_format TYPE = 'CSV' FIELD_DELIMITER = ','  PARSE_HEADER = TRUE;

                CREATE  OR REPLACE TABLE target_table5
                USING TEMPLATE (
                       SELECT ARRAY_AGG(OBJECT_CONSTRUCT(*))
                       WITHIN GROUP (ORDER BY ORDER_ID)
                       FROM TABLE(
                           INFER_SCHEMA(
                                 LOCATION=>'@internal_stage/inbound/data_85.csv',
                                 FILE_FORMAT=>'my_csv_format'
                                       )
                                )
                            );
TARGET_TABLE5 has been created


In [9]:
from configparser import ConfigParser


parser = ConfigParser()
parser.read('SNOWFLAKE_CREDS.cfg')
account = parser.get('my_api','account')
user = parser.get('my_api','user')
password = parser.get('my_api','password')
database = parser.get('my_api','database')
schema = parser.get('my_api','schema')
role = parser.get('my_api','role')
warehouse = parser.get('my_api','warehouse')

conn = snowflake.connector.connect( user = user ,password = password,account = account ,warehouse = warehouse ,
                                   database = database,schema = schema,role = role)

curs = conn.cursor()


curs.execute("PUT 'file://inbound/sales_2023.csv' '@internal_stage/inbound/sales_2024.csv'")